<a href="https://colab.research.google.com/github/muradpoly/Cat_Dog_classification/blob/main/Cat_Dog_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 #Importing all essential libraries
import torch
from torch import nn
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

from tqdm.auto import tqdm

In [ ]:
#Preparing pipeline for data augmentation
auqmentation_pipeline = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

#Importing the data
train_dataset = datasets.ImageFolder(
    root = r"/content/drive/MyDrive/CAT_DOG_DATASET/training_set/training_set",
    transform = auqmentation_pipeline
)

test_dataset = datasets.ImageFolder(
    root = r"/content/drive/MyDrive/CAT_DOG_DATASET/test_set/test_set",
    transform = transform
)
print(f"We have  {len(test_dataset)} samples in test data and {len(train_dataset)} as train samples")

We have  2033 samples in test data and 8030 as train samples


In [ ]:
# Turning the dataset into mini batches
from torch.utils.data import DataLoader

BATCH_SIZE = 32

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Dataloaders: {train_dataloader, test_dataloader}")
print(f"Length of train dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}")
print(f"Length of test dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}")

Dataloaders: (<torch.utils.data.dataloader.DataLoader object at 0x7f10d788ba30>, <torch.utils.data.dataloader.DataLoader object at 0x7f10d7889ea0>)
Length of train dataloader: 251 batches of 32
Length of test dataloader: 64 batches of 32


In [ ]:
import torch.nn as nn

class ClassModelV0(nn.Module):
    def __init__(self):
        super(ClassModelV0, self).__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(3, 6, 5, 1, 2),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(3, 3)
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(6, 16, 5, 1, 2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(3, 3)
        )

        self.block3 = nn.Sequential(
            nn.Conv2d(16, 16, 5, 1, 2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(3, 3)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(16 * 8 * 8, 84),
            nn.BatchNorm1d(84),
            nn.ReLU(),
            nn.Linear(84, 120),
            nn.BatchNorm1d(120),
            nn.ReLU(),
            nn.Linear(120, 2)
        )

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.classifier(x)
        return x

# Create an instance of the model
model_0 = ClassModelV0()

# Setup loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_0.parameters())

In [ ]:
# Define function to measure time
from timeit import default_timer as timer

def print_train_time(start: float, end: float):
  total_time = (end - start) / 60
  print(f"Train time : {total_time:.3f} minutes")
  return total_time


In [ ]:
# Creating the function to measure accuracy
def accuracy_fn(y_true, y_pred):
  correct = torch.eq(y_true, y_pred).sum().item()
  acc = (correct/len(y_pred)) * 100
  return acc

In [ ]:
def train_loop(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn):
    train_loss, train_acc = 0, 0

    for batch, (X, y) in enumerate(data_loader):

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y,
                                 y_pred=y_pred.argmax(dim=1)) # Go from logits -> pred labels

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

    # Calculate loss and accuracy per epoch and print out what's happening
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

In [ ]:
def test_loop(data_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              accuracy_fn):
    test_loss, test_acc = 0, 0

    model.eval() # put model in eval mode
    # Turn on inference context manager
    with torch.inference_mode():
        for X, y in data_loader:

            # 1. Forward pass
            test_pred = model(X)

            # 2. Calculate loss and accuracy
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y,
                y_pred=test_pred.argmax(dim=1) # Go from logits -> pred labels
            )

        # Adjust metrics and print out
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

In [ ]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn):
    train_loss, train_acc = 0, 0

    for batch, (X, y) in enumerate(data_loader):

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y,
                                 y_pred=y_pred.argmax(dim=1)) # Go from logits -> pred labels

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

    # Calculate loss and accuracy per epoch and print out what's happening
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

def test_step(data_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              accuracy_fn):
    test_loss, test_acc = 0, 0

    model.eval() # put model in eval mode
    # Turn on inference context manager
    with torch.inference_mode():
        for X, y in data_loader:

            # 1. Forward pass
            test_pred = model(X)

            # 2. Calculate loss and accuracy
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y,
                y_pred=test_pred.argmax(dim=1) # Go from logits -> pred labels
            )

        # Adjust metrics and print out
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")


In [ ]:
torch.manual_seed(99)

# Measure time
from timeit import default_timer as timer
train_time_start = timer()

epochs = 10
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n---------")
    train_loop(data_loader=train_dataloader,
        model=model_0,
        loss_fn=loss_fn,
        optimizer=optimizer,
        accuracy_fn=accuracy_fn
    )
    test_loop(data_loader=test_dataloader,
        model=model_0,
        loss_fn=loss_fn,
        accuracy_fn=accuracy_fn
    )

train_time_end = timer()
total_train_time_model_0 = print_train_time(start=train_time_start,
                                            end=train_time_end)


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0
---------
Train loss: 0.61053 | Train accuracy: 65.85%
Test loss: 0.59558 | Test accuracy: 68.01%

Epoch: 1
---------
Train loss: 0.56287 | Train accuracy: 70.50%
Test loss: 0.52464 | Test accuracy: 74.14%

Epoch: 2
---------
Train loss: 0.51479 | Train accuracy: 74.39%
Test loss: 0.50129 | Test accuracy: 75.05%

Epoch: 3
---------
Train loss: 0.47304 | Train accuracy: 77.37%
Test loss: 0.50856 | Test accuracy: 75.46%

Epoch: 4
---------
Train loss: 0.45109 | Train accuracy: 78.03%
Test loss: 0.45570 | Test accuracy: 78.48%

Epoch: 5
---------
Train loss: 0.43148 | Train accuracy: 79.61%
Test loss: 0.46642 | Test accuracy: 77.21%

Epoch: 6
---------
Train loss: 0.40801 | Train accuracy: 81.32%
Test loss: 0.43902 | Test accuracy: 79.26%

Epoch: 7
---------
Train loss: 0.38947 | Train accuracy: 82.36%
Test loss: 0.42879 | Test accuracy: 79.50%

Epoch: 8
---------
Train loss: 0.37297 | Train accuracy: 82.85%
Test loss: 0.41310 | Test accuracy: 81.96%

Epoch: 9
---------
Train los